##Installing and Importing

In [ ]:
!pip install xmltodict
!pip install beautifulsoup4
!pip install requests
!pip install pystemmer
!pip install gdown
!pip install fuzzywuzzy
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 559 kB 7.2 MB/s 
  Created wheel for pystemmer: filename=PyStemmer-2.0.1-cp37-cp37m-linux_x86_64.whl size=426264 sha256=97048fda1e3d73bafd62acb97c50ded8eb5e9403688dab077738d1539c7e3600
  Stored in directory: /root/.cache/pip/wheels/30/6d/40/0d17a498c5009922dbb3ddaca3d3652387ba94cc96142001f0
Successfully built pystemmer
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, 

In [ ]:
!gdown --id 1YSyGX_oS0zusjv9Mw8ub0ZdUC--18kvo 

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1YSyGX_oS0zusjv9Mw8ub0ZdUC--18kvo
To: /content/tokens.json
100% 39.4M/39.4M [00:00<00:00, 41.8MB/s]


In [ ]:
!gdown --id 10xmDYwtcjfNCvPLIt9bHaPbMQTAU2rM9

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=10xmDYwtcjfNCvPLIt9bHaPbMQTAU2rM9
To: /content/array.json
100% 190M/190M [00:02<00:00, 75.4MB/s]


In [ ]:
!gdown --id 1DWazm5rnGBb7zQmbRhgCYDzYNwlmLyvQ
!gdown --id 15QnJJeUAVrLG1x84u2CAnf6fNkXMew98

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1DWazm5rnGBb7zQmbRhgCYDzYNwlmLyvQ
To: /content/datasetFile.txt
100% 4.03M/4.03M [00:00<00:00, 158MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=15QnJJeUAVrLG1x84u2CAnf6fNkXMew98
To: /content/combined.pickle
100% 452k/452k [00:00<00:00, 33.0MB/s]


In [ ]:
# !wget https://raw.githubusercontent.com/neechalkaran/Tamil-corpus/master/Most%20used%20Rootwords.csv

In [ ]:
from bs4 import BeautifulSoup
import requests
import xmltodict
import json
import re
import os
import os.path
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
import Stemmer
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

## Functions declaration

In [ ]:
def getLinks():
  r = requests.get("https://venmurasu.in/sitemap.xml")
  xml = r.text
  o = xmltodict.parse(xml)
  obj = json.loads(json.dumps(o))
  arr = []

  for i in obj["urlset"]["url"]:
    arr.append(i["loc"])

  arr.sort()
  return arr

In [ ]:
def normalize(txt):
  regex = r"[^\u0b80-\u0bff]"
  return re.sub(regex," ",txt)

In [ ]:
def scrapeSite(url):
  html = urllib.request.urlopen(url)
  htmlParse = BeautifulSoup(html, 'html.parser')
  arr = []
  for para in htmlParse.find_all("p"):
    arr.append(normalize(para.get_text()))
  #print(arr)
  return arr

In [ ]:
def tokenize(arr):
  tokenizer1 = Tokenizer(num_words=10**9)
  tokenizer1.fit_on_texts(arr)
  word_index = tokenizer1.word_index
  return word_index

In [ ]:
def scrapeAllSites():

  if os.path.exists("array.json"):
    with open("array.json") as f:
      json_object = json.load(f)
      return json_object["arr"]

  arr = []
  siteContent = {}
  n = 0
  err_sites = []
  links = getLinks()
  linksSize = len(links)
  for i in links:
    n += 1
    try:
      cont = scrapeSite(i)
      arr.extend(cont)
    except:
      err_sites.append(i)
    print("\r\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b\b{}/{}".format(n, linksSize))
  print("Error sites : ", err_sites)
  
  with open("array.json", "w") as f:
    d = {
        "arr" : arr
    }
    json_object = json.dumps(d, indent = 2)
    f.write(json_object)
  
  return arr

In [ ]:
def tokenizeAll(arr):

  if os.path.exists("tokens.json"):
    with open("tokens.json") as f:
      json_object = json.load(f)
    return json_object

  d = dict(tokenize(arr))
  with open("tokens.json","w") as f:
    json.dump(d, f)
  return d

## Testing

In [ ]:
arr = getLinks()
print(len(arr), "pages")

1937 pages


In [ ]:
temp = scrapeSite(arr[-20])
#print(arr[-20], len(temp), *temp, sep="\n\n")
temp_d = tokenize(temp)
print(list(temp_d))

['அவன்', 'என்று', 'அதன்', 'அவர்கள்', 'அது', 'தங்கள்', 'மேல்', 'எழுந்து', 'என்றனர்', 'அவர்', 'வந்து', 'அனலோன்', 'கொண்டு', 'ஆம்', 'திரியை', 'நோக்கி', 'அங்கே', 'என', 'மட்டுமே', 'அவர்களின்', 'எங்கள்', 'நந்தவாசுகி', 'எரி', 'தன்', 'அன்னை', 'என்றாள்', 'எழுந்தது', 'இந்திரன்', 'இந்திரனின்', 'அமர்ந்து', 'இறங்கி', 'போர்', 'சென்று', 'தட்சநாகர்கள்', 'மடிவது', 'என்', 'நாம்', 'அறிக', 'தன்னை', 'எழுந்த', 'அமைந்த', 'உடலின்', 'காற்று', 'அவனை', 'தெய்வங்கள்', 'என்றது', 'நின்று', 'இந்த', 'மீண்டும்', 'அவர்களை', 'தட்சர்', 'என்றார்', 'முன்', 'ஏறி', 'வணங்கி', 'நம்', 'புகுந்து', 'இதோ', 'இனி', 'மைந்தரே', 'அதை', 'அந்த', 'மூதன்னை', 'எட்டு', 'கொண்டது', 'கணமே', 'வெண்முட்டை', 'ஓட்டை', 'சீறி', 'உடல்', 'விரிந்த', 'நாகங்கள்', 'இந்திரனுடன்', 'இந்திரவில்', 'நடுவே', 'அனல்', 'நூறு', 'அவள்', 'இந்திரனை', 'நெருப்பு', 'எழும்', 'நாள்', 'எவரும்', 'போர்களை', 'விழிதிறந்து', 'போரில்', 'சென்ற', 'அறைந்து', 'எனும்', 'பறக்கும்', 'தட்சர்களின்', 'பன்னிரண்டு', 'ஏழு', 'ஊனும்', 'வாழ்ந்த', 'கரையில்', 'மலைச்சரிவில்', 'கொண்டுவந்து', 'தட்சநாகர்கள

## Final Indexing

In [ ]:
arr = scrapeAllSites()
word_index = tokenizeAll(arr)
#print(word_index)
len(word_index)

449239

##Stemming using PyStemmer

In [ ]:
# coll = set()
# n = len(word_index)
# i = 0
# stemmer = Stemmer.Stemmer('tamil')
# for word in list(word_index):
#   i += 1
#   print("\r\b\b\b\b\b\b\b{:.3f}% {}/{}".format(i*100/n,i,n), end="")
#   coll.add(stemmer.stemWord(word))

In [ ]:
# len(coll)

In [ ]:
# print(tuple(coll2)[:15])

In [ ]:
stemmer = Stemmer.Stemmer('tamil')
for word in list(word_index)[50:100]:
  print(word," --> ",stemmer.stemWord(word))

பின்னர்  -->  பின்
நின்று  -->  ந்
தங்கள்  -->  தங்
அவர்களின்  -->  அவர்
உங்கள்  -->  
இன்று  -->  இன்று
உள்ளே  -->  உள்
அங்கு  -->  கு
பீமன்  -->  பீமன்
அல்ல  -->  அல்ல
மேல்  -->  மேல்
நம்  -->  நம்
வெளியே  -->  வெளி
ஏன்  -->  ஏன்
முன்  -->  முன்
சொன்னார்  -->  சொன்
அப்பால்  -->  பால்
இந்த  -->  இ
எனக்கு  -->  என
இங்கே  -->  கே
சொல்  -->  சொல்
போல  -->  போல
அவர்களை  -->  அவர்
இருந்து  -->  இரு
கொண்டு  -->  கொண்டு
துரியோதனன்  -->  துரியோ
அவளை  -->  அவளை
யாதவர்  -->  யாத
வந்த  -->  வ
முகம்  -->  முகம்
என்றும்  -->  எல்
சற்று  -->  சற்று
கொண்ட  -->  கொண்ட
இருந்தது  -->  இரு
ஓர்  -->  ஓர்
அரசே  -->  அரசே
அரசி  -->  அரசி
என்னும்  -->  என்
பெரிய  -->  பெரி
பின்  -->  பின்
மட்டும்  -->  மடு
எழுந்த  -->  எழு
தோன்றியது  -->  தோன்றி
தன்னை  -->  தன்
அமர்ந்து  -->  அமர்
உள்ளம்  -->  உள்ளம்
எடுத்து  -->  எடு
அத்தனை  -->  தனை
இனி  -->  இனி
பின்னால்  -->  பின்


In [ ]:
stemmer = Stemmer.Stemmer('tamil')
for word in list(word_index)[:50]:
  print(word," --> ",stemmer.stemWord(word))

என்று  -->  என்று
என்றான்  -->  எல்
அவன்  -->  அவன்
நான்  -->  நான்
என்றார்  -->  எல்
அவர்  -->  அவர்
என  -->  என
அவள்  -->  அவள்
அது  -->  அது
என்  -->  என்
ஒரு  -->  ஒரு
என்றாள்  -->  எல்
தன்  -->  தன்
அதை  -->  அதை
அவர்கள்  -->  அவர்
ஆம்  -->  ஆம்
ஆனால்  -->  ஆன்
நோக்கி  -->  நோக்கி
வந்து  -->  வந்
என்ன  -->  என்ன
சென்று  -->  ச்
எழுந்து  -->  எழு
நீ  -->  நீ
இல்லை  -->  இல்
அந்த  -->  அ
மீண்டும்  -->  மீண்
நாம்  -->  நாம்
அவனை  -->  அவனை
அதன்  -->  அதன்
அவை  -->  அவை
இங்கு  -->  கு
இது  -->  இது
அங்கே  -->  கே
மட்டுமே  -->  மடு
இளைய  -->  இளை
நீங்கள்  -->  நீங்
மேலும்  -->  மேல்
அர்ஜுனன்  -->  அர்ஜு
திரும்பி  -->  திரும்
உடல்  -->  உடல்
என்னை  -->  என்
மெல்ல  -->  மெல்ல
ஒவ்வொரு  -->  ஒவ்வொரு
இரு  -->  இரு
சொன்னான்  -->  சொன்
ஒன்று  -->  ஒன்று
உன்  -->  உன்
அருகே  -->  அர்
கர்ணன்  -->  கர்ணன்
அவரை  -->  அவரை


## Snowball from Scratch

In [ ]:
# !wget https://github.com/rdamodharan/tamil-stemmer/blob/master/tamil-stem_Unicode.sbl tamil.sbl

In [ ]:
# !pip install snowballstemmer
# import snowballstemmer
# stemmer = Stemmer.Stemmer('tamil')
# #print(stemmer.algorithms())

## Custom Stemming Function(s)

**Reference PDF : [*here*](http://www.ijcset.com/docs/IJCSET17-08-06-023.pdf)**

In [ ]:
def hasQuestionSuffix(word):
  return (word[-1] in ["ா", "ே", "ோ"])

def fixQuestionSuffix(word):
  return word[:-1] + "்" 

def hasConjuctionSuffix(word):
  return (word[-3:] == "ும்")

def fixConjuctionSuffix(word):
  return word[:-3] + "்" 
  #return word[:-2]

def hasPluralSuffix(word):
  return (word[-5:] == "ங்கள்") or (word[-3:] == "கள்") or word[-4:] == "றனர்" or word[-3:] == "னர்" 

def fixPluralSuffix(word):
  if word[-5:] == "ங்கள்":
    return word[:-5] + "ம்"
  elif word[-3:] == "கள்":
    return word[:-3]
  elif word[-4:] == "றனர்":
    return (word[:-3]+ "ு")
  elif word[-3:] == "னர்":
    return word[:-3]

#implement from here

def hasImperativeSuffix(word):
   return (word[-1] in ["ி"])

def fixImperativeSuffix(word):
  #logic changed here
  #return word[:-1]
  return word[:-2]

def hasTenseSuffix(word):
    return (word[-6:] == "கின்றன") or (word[-3:] in ["ேன்"]) or (word[-16:-6]=="கொண்டிருக்")

def fixTenseSuffix(word):
  if (word[-4:]=="வேன்"):  #for future tense
     return word[:-4]
  if (word[-3:] in ["ேன்"]):
    return word[:-5] + "ு"
  if (word[-6:] == "கின்றன"):  # present
     return word[:-6]
  if(word[-16:-6]=="கொண்டிருக்"):
     return word[:-18]
  

def hasCommonWordSuffix(word):
  return  "ில்லாத" in word 

def fixCommonWordSuffix(word):
  return word[:-6]+ "்"

#வேற்றுமை உருப்புகள்
def hasCaseSuffix(word):
  return (word[-4:-2] == word[-2:]) or (word[-6:] in ["த்தில்"]) or (word[-4:] in ["ிடம்", "த்த்"]) or (word[-1]=='ை') or (word[-2:] in ["த்","ந்"]) or  (word[-4:] in ["த்து","ந்து","ந்த"] or word[-3:] in ["ந்த"]) or (word[-3:] == "ின்") or (word[-1:]== "ு")

#வேற்றுமை உருப்புகள்
def fixCaseSuffix(word):
  if(word[-4:-2] == word[-2:]):
    return word[:-2]
  if(word[-1:]== "ு"):
    if(word[-2]==word[-3]):
        return word[:-3]
    else:
        return word[:-2]
  if(word[-6:] in ["த்தில்"]):
    return word[:-6]+"ம்"
  if (word[-4:] in ["த்து","ந்து","ந்த"] or word[-3:] in ["ந்த"]):
    if(word[-3:] in ["ந்த"]):
      return word[:-3]
    return word[:-4]
  if word[-4:] == "த்த்":
    return word[:-4] + "ம்"
  if(word[-4:] == "ிடம்"):
    return word[:-4]+'்'
  if(word[-1]=='ை'):
    return word[:-1]+'்'
  if(word[-2:] in ["த்","ந்"]):  # to remove last extra mei eluthu
    return word[:-2]
  if word[-3:] == "ின்":
    return word[:-3] + '்'
  return word

In [ ]:
def stem(word):

  iteration = 0
  temp = word.replace('ஂ', '்')
  
  while temp and len(temp) > 4:

    iteration += 1
    # print("Beginning of iteration {} : {}".format(iteration, temp))

    if hasQuestionSuffix(temp):
      tempStore = temp
      temp = fixQuestionSuffix(temp)
      if(tempStore == temp):
        break

    elif hasConjuctionSuffix(temp):
      tempStore = temp
      temp = fixConjuctionSuffix(temp)
      if(tempStore == temp):
        break

    elif hasCaseSuffix(temp):
      tempStore = temp
      temp = fixCaseSuffix(temp)
      if(tempStore == temp):
        break

    elif hasPluralSuffix(temp):
      tempStore = temp
      temp = fixPluralSuffix(temp)
      if(tempStore == temp):
        break

    elif hasImperativeSuffix(temp):
      tempStore = temp
      temp = fixImperativeSuffix(temp)
      if(tempStore == temp):
        break

    elif hasTenseSuffix(temp):
      tempStore = temp
      temp = fixTenseSuffix(temp)
      if(tempStore == temp):
        break
      

    elif hasCommonWordSuffix(temp):
      tempStore = temp
      temp = fixCommonWordSuffix(temp)
      if(tempStore == temp):
        break

    else:
      break
    
  return temp

In [ ]:
#word = "அவனும்"
#word = "அவனா"
#word = "அவனோ"
#word = "அவன்"
#word = "மரங்கள்"
#word = "அரசனிடம்"
#word = "அவளை"
#word = "மரத்தை"
#word = "செய்வேன்"
#word = "பாடினேன்"
#word = "செய்வேன்"
#word = "சகுனி"
#word = "கொண்டிருக்கிறாள்"
#word = "படைத்து"
#word = "அடைந்து"
# word = "அரசனும்"
# word = "அவர்களின்" 
#word = "அவனில்லாத"
# word = "அமைந்த"
# word="என்னும்"
# word="இருந்து"

word="என்று"
print(list(word))
print("Final : ", stem(word))

['எ', 'ன', '்', 'ற', 'ு']
Final :  என்


In [ ]:
print(ord('ஂ'), ord('்'))

2946 3021


In [ ]:
print(list(temp_d)[30:60])
print(len(list(temp_d)[30:60]))
print()

for i in list(temp_d)[30:60]:
  print(i, end=" ")
  print(stem(i))

['இறங்கி', 'போர்', 'சென்று', 'தட்சநாகர்கள்', 'மடிவது', 'என்', 'நாம்', 'அறிக', 'தன்னை', 'எழுந்த', 'அமைந்த', 'உடலின்', 'காற்று', 'அவனை', 'தெய்வங்கள்', 'என்றது', 'நின்று', 'இந்த', 'மீண்டும்', 'அவர்களை', 'தட்சர்', 'என்றார்', 'முன்', 'ஏறி', 'வணங்கி', 'நம்', 'புகுந்து', 'இதோ', 'இனி', 'மைந்தரே']
30

இறங்கி இறங்
போர் போர்
சென்று சென்
தட்சநாகர்கள் தட்சநாகர்
மடிவது மடிவ
என் என்
நாம் நாம்
அறிக அறிக
தன்னை தன்
எழுந்த எழு
அமைந்த அமை
உடலின் உடல்
காற்று காற்
அவனை அவனை
தெய்வங்கள் தெய்வம்
என்றது என்ற
நின்று நின்
இந்த இந்த
மீண்டும் மீண்ட்
அவர்களை அவர்
தட்சர் தட்சர்
என்றார் என்றார்
முன் முன்
ஏறி ஏறி
வணங்கி வணங்
நம் நம்
புகுந்து புகு
இதோ இதோ
இனி இனி
மைந்தரே மைந்தர்


## Refining Root Words

In [ ]:
# import pandas as pd

# dataset_fab = pd.read_csv('Fabricius Dictionary.csv')
# dataset_Kad = pd.read_csv('Kadirvelu Dictionary.csv')
# dataset_Mc = pd.read_csv('McAlpin Dictionary.csv')
# dataset_mos = pd.read_csv('Most used Rootwords.csv')
# dataset_wik = pd.read_csv('Wiktionary words.csv')
# dataset_win= pd.read_csv('Winslow.csv')
# dataset_idd= pd.read_csv('lddttam.csv')

In [ ]:
# dataset_fab = dataset_fab.iloc[ : , :1]  # first columns
# dataset_fab = dataset_fab[dataset_fab.columns[0]].tolist()

In [ ]:
# dataset_idd = dataset_idd.iloc[ : , :1] 
# dataset_idd = dataset_idd[dataset_idd.columns[0]].tolist()

In [ ]:
# dataset_Kad = dataset_Kad.iloc[: , :1]
# dataset_Kad = dataset_Kad[dataset_Kad.columns[0]].tolist()

In [ ]:
# dataset_Mc = dataset_Mc.iloc[: , :1]
# dataset_Mc = dataset_Mc[dataset_Mc.columns[0]].tolist() 

In [ ]:
# dataset_mos = dataset_mos.iloc[: , :1]
# dataset_mos = dataset_mos[dataset_mos.columns[0]].tolist()

In [ ]:
# dataset_wik = dataset_wik.iloc[:,:1]
# dataset_wik = dataset_wik[dataset_wik .columns[0]].tolist()

In [ ]:
# dataset_win = dataset_win.iloc[1: , :1]

# dataset_win = dataset_win[dataset_win.columns[0]].tolist()

In [ ]:
# import re

In [ ]:
# text = "விஷ்ணுAA#$1234567890!\@\#\$\%\^\&\*\(\)\{\}\:\"\;\'\[\]\<\>\?\,\.\")qwertyuiopasdfghjklzxcvbnm   `~/*-+ 1234567890"
# print(text)
# print(re.sub(r"[^\u0B80-\u0BFF]+", '', text))

In [ ]:
# dataset_win[:10]

In [ ]:
# print(bool(re.search(r"[^\u0B80-\u0BFF]+", 'அஆ')))

In [ ]:
# datasets = [ dataset_idd, dataset_Kad, dataset_Mc, dataset_mos, dataset_wik, dataset_win  ]
# words = set()

# for i in datasets:
#   for index, values in enumerate(i):
#     values = str(values)
#     if(len(values) == 1):
#       i.remove(values)
#     else:
#       if re.search(r"[^\u0B80-\u0BFF]+", values):
#         try:
#           i.remove(values)
#         except:
#           print(values)
#       elif(len(values) == 1):
#           i.remove(values)
#       else:
#         words.add(values)

In [ ]:
# datasetFile = open('datasetFile.txt','w+')

In [ ]:
# word_list = list(words)
# word_list.sort()
# word_list[:10]

In [ ]:
# datasetFile.write("\n".join(word_list))
# datasetFile.close()

In [ ]:
# file = open('datasetFile.txt', 'r')
# for i in file.readlines()[:20]:
#   print(i)

In [ ]:
rootwords = []
chdf = open('datasetFile.txt')
for r in chdf.readlines():
        rootwords.append(r.strip())

In [ ]:
# rootwords[:10]

## Word Spliting

In [ ]:
# chdf = pd.read_csv('Most used Rootwords.csv')
# chdf['derivation'] = chdf.derivation.str.split(',')
# chdf.head()
# l = []
# for i,r in chdf.iterrows():
#     for j in r['derivation']:
#         l.append([j,r['Root']])

# dfpr = pd.DataFrame(l,columns = ['Words','Root'])
# dfpr.head(100)

In [ ]:
# rootwords = chdf['Root'].unique().tolist()

In [ ]:
print(rootwords[:100])

['ஃபரூக்', 'ஃபாரஸ்ட்', 'ஃபிரெஞ்சு', 'ஃபேன்', 'ஃபேவரைட்', 'ஃபேஷன்', 'ஃபேஸ்புக்', 'ஃபோன்', 'அஃகடி', 'அஃகம்', 'அஃகரம்', 'அஃகல்', 'அஃகான்', 'அஃகாமை', 'அஃகியஐ', 'அஃகியஔ', 'அஃகியதனிநிலை', 'அஃகியமஃகான்', 'அஃகியவறிவு', 'அஃகு', 'அஃகுதல்', 'அஃகுல்லி', 'அஃகுள்', 'அஃகுவஃகெனல்', 'அஃகேனம்', 'அஃதான்று', 'அஃதாவது', 'அஃதி', 'அஃது', 'அஃதே', 'அஃதை', 'அஃபோதம்', 'அஃறிணை', 'அஆ', 'அஇஅதிமுக', 'அஇவனம்', 'அக', 'அகஇருள்', 'அககந்தம்', 'அககுரு', 'அககுருக்கி', 'அககுழா', 'அகக்கடவுள்', 'அகக்கணு', 'அகக்கண்', 'அகக்கமலம்', 'அகக்கரணம்', 'அகக்கர்ணம்', 'அகக்களி', 'அகக்காழன', 'அகக்காழ்', 'அகக்கூடு', 'அகக்கூத்து', 'அகக்கூத்துக்கை', 'அகங்கரம்', 'அகங்கரி', 'அகங்கரித்தல்', 'அகங்கரிப்பு', 'அகங்களிக்க', 'அகங்களிப்பு', 'அகங்காரகிரந்தி', 'அகங்காரக்கிரந்தி', 'அகங்காரசைதன்னியவாதி', 'அகங்காரத்திரயம்', 'அகங்காரம்', 'அகங்காரவாதி', 'அகங்காரவிர்த்தி', 'அகங்காரி', 'அகங்காழ்', 'அகங்கிருதி', 'அகங்கை', 'அகசன்', 'அகசம்', 'அகசரிப்பு', 'அகசியகாரன்', 'அகசியக்காரன்', 'அகசியக்கூத்து', 'அகசியம்', 'அகசு', 'அகசை', 'அகச்சட்டகம்', 'அகச்சத்தாதுவித்தசமாதி

In [ ]:
print(len(rootwords))

128215


In [ ]:
# !pip install numba
# !pip install cudatoolkit

In [ ]:
def fuzzymatch(string, master = rootwords):
  score = 0
  arr = []
  d = {}
  for (i,j) in process.extract(string, master, limit = 10):
    ratio = fuzz.partial_ratio(i,string)
    if ratio >= 85:
      score += 1
      arr.append(i)
      d[i] = ratio
  return (score, arr, d)

In [ ]:
fuzzymatch("பெருஞ்சாலை")

(0, [], {})

## Master Output

In [ ]:
from random import sample

arr = []
cols = ["original", "pystemmer : f(x)", "custom : g(x)", "smallest length", "f(g(x))", "g(f(x))", "f(f(x))", "g(g(x))", "Score", "word splitted array"]
#temp_word_index = sample(list(word_index),200)

n = 100
i = 0
stemmer = Stemmer.Stemmer('tamil')
for word in list(word_index)[:100]:
  temp = [word, stemmer.stemWord(word), stem(word)]
  f1 = temp[1] if len(temp[1]) < len(temp[2]) else temp[2]
  f2 = temp[1] if len(temp[1]) >= len(temp[2]) else temp[2]
  if len(f1) <= 2:
    temp.append(f2)
  else:
    temp.append(f1)
  
  temp.append(stemmer.stemWord(temp[2]))
  temp.append(stem(temp[1]))
  temp.append(stemmer.stemWord(temp[1]))
  temp.append(stem(temp[2]))
  fuzzy_temp = fuzzymatch(word)
  temp.append(fuzzy_temp[0])
  temp.append(fuzzy_temp[1])
  arr.append(temp)
  i += 1
  print("\r\b\b\b\b\b\b\b{:.3f}% {}/{}".format(i*100/n,i,n), end="")

df = pd.DataFrame(data = arr, columns = cols)

100.000% 100/100

## Results as of now

In [ ]:
df.to_csv("resuls.csv")

In [ ]:
# exact result from pystemmer and custom algo
same = df[df['custom : g(x)'] ==  df['pystemmer : f(x)']]
print(same.shape[0], "=>", same.shape[0]*100/449239,"%")
same.sample(20)

58 => 0.012910722354915758 %


,original,pystemmer : f(x),custom : g(x),smallest length,f(g(x)),g(f(x)),f(f(x)),g(g(x)),Score,word splitted array
60,மேல்,மேல்,மேல்,மேல்,மேல்,மேல்,மேல்,மேல்,0,[]
6,என,என,என,என,என,என,என,என,10,"[என, எனா, எனை, என், உஸ்என்றோட்ட, எனக்குரியவன்,..."
64,முன்,முன்,முன்,முன்,முன்,முன்,முன்,முன்,0,[]
13,அதை,அதை,அதை,அதை,அதை,அதை,அதை,அதை,1,[அதை]
26,நாம்,நாம்,நாம்,நாம்,நாம்,நாம்,நாம்,நாம்,2,"[நாம, நாம்]"
50,பின்னர்,பின்,பின்,பின்,பின்,பின்,பின்,பின்,2,"[பின்னரை, பின்னர்]"
82,கொண்ட,கொண்ட,கொண்ட,கொண்ட,கொண்ட,கொண்ட,கொண்ட,கொண்ட,0,[]
23,இல்லை,இல்,இல்,இல்,இல்,இல்,இல்,இல்,3,"[இல்ல, இல்லை, இல]"
22,நீ,நீ,நீ,நீ,நீ,நீ,நீ,நீ,1,[நீ]
46,உன்,உன்,உன்,உன்,உன்,உன்,உன்,உன்,6,"[உன், உன்னதக்கோரி, உன்னதமாயிருக்க, உன்னதம், உன..."


In [ ]:
df.sample(50)

,original,pystemmer : f(x),custom : g(x),smallest length,f(g(x)),g(f(x)),f(f(x)),g(g(x)),Score,word splitted array
34,இளைய,இளை,இளைய,இளை,இளை,இளை,இளை,இளைய,8,"[இளைய, இளை, இளையபெருமாள், இளையமரக்கால், இளையவன..."
41,மெல்ல,மெல்ல,மெல்ல,மெல்ல,மெல்ல,மெல்ல,மெல்ல,மெல்ல,2,"[மெல்ல, மெல்லி]"
33,மட்டுமே,மடு,மட்,மட்,மட்,மடு,மடு,மட்,1,[மட்டும்]
78,வந்த,வ,வந்த,வந்த,வ,வ,வ,வந்த,3,"[வந்தி, வந்து, வந்தை]"
69,இங்கே,கே,இங்க்,இங்க்,இங்,கே,கே,இங்க்,1,[இங்கே]
57,அங்கு,கு,அங்,அங்,அங்,கு,கு,அங்,1,[அங்கு]
15,ஆம்,ஆம்,ஆம்,ஆம்,ஆம்,ஆம்,ஆம்,ஆம்,1,[ஆம்]
61,நம்,நம்,நம்,நம்,நம்,நம்,நம்,நம்,8,"[நம, நம், அத்தியாசநம், அத்தியாயநம், அநம், அநுக..."
5,அவர்,அவர்,அவர்,அவர்,அவர்,அவர்,அவர்,அவர்,2,"[அவர், அவ]"
12,தன்,தன்,தன்,தன்,தன்,தன்,தன்,தன்,5,"[தன், அகண்டிதன், அகதன், அகத்தன், அகப்பேய்ச்சித..."
